In [58]:
import geopandas as gpd


In [59]:
DEFAULT_CRS = "EPSG:4326"

# lendos shapefiles



In [75]:
hex_iptu = gpd.read_file("../../data/generate/WGS84_hex_iptu_2020.gpkg")
hex_contruco = gpd.read_file("../../data/generated/WGS84_hex_construcao_open_buildings_2023.gpkg")
hex_cnefe = gpd.read_file("../../data/hex_agg_cnef_sp.gpkg")
hex_censo = gpd.read_file("../../data/generated/WGS84_hex_censo_2022.gpkg")
#hex_favelas = gpd.read_file("../../data/WGS84_hex_favelas.gpkg")
hex_fcu = gpd.read_file("../../data/generated/WGS84_hex_fcu.gpkg")

hex_osm = gpd.read_file("../../data/generated/WGS84_hex_sp_osm.gpkg")
hex_declividade = gpd.read_file("../../data/raw/WGS84_hex_declividade_sp.gpkg")


In [76]:
hex_censo.head()

,id_hex,domicilios_agua_rede_geral_n_sum,domicilios_esgoto_rede_geral_n_sum,domicilios_lixo_coletado_n_sum,domicilios_n_sum,area_poly_urbano_sum,area_poly_rural_sum,Households connected to water supply network (%),Households connected to sewerage system (%),Households solid waste collection (%),geometry
0,8aa8100c3257fff,86.0,86.0,86.0,86.0,18000.0,0.0,1.000000,1.000000,1.000000,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5..."
1,8aa8100d2447fff,112.0,112.0,111.0,112.0,18008.0,0.0,1.000000,1.000000,0.991071,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5..."
2,8aa8100e1637fff,43.0,42.0,42.0,43.0,17996.0,0.0,1.000000,0.976744,0.976744,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5..."
3,8aa8100f0827fff,401.0,385.0,402.0,402.0,17999.0,0.0,0.997512,0.957711,1.000000,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6..."
4,8aa81077552ffff,29.0,26.0,29.0,29.0,18007.0,0.0,1.000000,0.896552,1.000000,"POLYGON ((-46.59688 -23.46650, -46.59760 -23.4..."


In [77]:
def set_default_crs(gdf):
    return gdf.to_crs(DEFAULT_CRS)

In [78]:
# renaming some columns
hex_cnefe.rename(columns={'esp_1': 'Number of private residences'}, inplace=True) #TODO rename in the hex agreggation file 


hex_declividade.rename(columns={'DCL_mean': 'Average slope'}, inplace=True)

In [86]:
hex_iptu = set_default_crs(hex_iptu)
hex_contruco = set_default_crs(hex_contruco)
hex_cnefe = set_default_crs(hex_cnefe)
hex_census = set_default_crs(hex_censo)
hex_osm = set_default_crs(hex_osm)
hex_declividade = set_default_crs(hex_declividade)

In [87]:
print(hex_census.shape)
print(hex_cnefe.shape)

(54855, 11)
(105505, 31)


# Agrupando arquivos

In [88]:
hex_f = hex_censo[['id_hex', 'Households connected to water supply network (%)', 'Households connected to sewerage system (%)', 'Households solid waste collection (%)']].join(hex_iptu.set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_cnefe[['id_hex', 'Number of private residences']].set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_fcu[['id_hex', 'is_fcu', 'prop_ocupado']].set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_osm.drop(['geometry'], axis=1).set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_declividade.drop(['geometry'], axis=1).set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_contruco.drop(['geometry'], axis=1).set_index("id_hex"), how='left', on='id_hex')



# Transformações finais
- fill all nan

In [82]:
#hex_f["hex_fid"] = hex_f["fid"].astype(int) #geopandas as a bug with fid https://github.com/geopandas/geopandas/issues/1035
#hex_f.drop("fid", inplace=True, axis=1)

In [89]:
hex_f.fillna(0, inplace=True)

In [90]:
hex_f[hex_f['id_hex'] == '8aa8100c0747fff']

,id_hex,Households connected to water supply network (%),Households connected to sewerage system (%),Households solid waste collection (%),qnt_unidades_residencias_sum,qnt_unidades_residencias_mean,qnt_unidades_residencias_median,qnt_unidades_residencias_std,qnt_unidades_residencias_var,geometry,...,DCL_median,DCL_stdev,area_sum,Average building area,area_median,Standard deviation of building area,area_var,Number of buildings areas,area_hex,area_const
35491,8aa8100c0747fff,0.999219,1.0,0.997658,1503.0,46.966267,18.007929,58.99313,3480.189412,"POLYGON ((-46.62875 -23.55391, -46.62947 -23.5...",...,15.794018,3.530481,6456.330991,94.946044,75.439379,88.322568,7800.87604,68.0,18001.957827,0.358646


In [91]:
hex_f = gpd.GeoDataFrame(hex_f, crs=DEFAULT_CRS, geometry=hex_f['geometry'])

In [92]:
hex_f.to_file("../../data/generate/WGS84_hex_sp_agg_2022.gpkg", driver="GPKG")